<a href="https://colab.research.google.com/github/ethanherrera/info159/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 5: Relation Extraction

### Due Date: April 10 (11:59pm)

**Total Points: 100 points**

**Make a copy** of this notebook and work only on your copy, so that your work is retained. Make sure your copy is named `HW5.ipynb`.  

# Introduction

In this assignment, we'll examine how one could use **bootstrapping** to extract relations between entities from a dataset.

Imagine you have been appointed to manage a large and ecologically diverse preserve. Here, the wifi is shaky, and the cell service is non-existent. It'll be a nice respite from trudging through endless midterms at Berkeley.

However, this morning, a group of long-tailed macques ambushed you and stole your paper records detailing a taxonomy of the animals. You're left in a troubling situation, as without this taxonomy, it's tough to know which animals are which types and properly care for them. What are giraffes? Cephalopods? You're not too sure.

The zoologists are on vacation, and have only left behind a collection of long-form notes describing the animals. Your only hope is to use relation extraction methods from INFO 159/259 to restructure these notes and get a head start on rebuilding the stolen taxonomy.

Your knowledge of animals is pretty flimsy. Luckily, you recall from Jurafsky & Martin Chapter 20 the following algorithm for bootstrapping:

```
function BOOTSTRAP(Relation_R) returns new_relation_tuples
  tuples ← Gather a set of seed tuples that have relation R
  iterate
    sentences ← find sentences that contain entities in tuples
    patterns ← generalize the context between and around entities in sentences
    new_pairs ← use patterns to identify more tuples
    new_pairs ← new_pairs with high confidence
    tuples ← tuples + new_pairs
  return tuples
```

In this homework, we'll demonstrate the affordances and caveats of this algorithm by walking through one iteration. We'll gather phrases that are suggestive of *is-a* relations between animals and their types, and recover new hyponym-hypernym pairs to save the day.  

### Remember:

**Only** modify text or code in between `### BEGIN SOLUTION` and `### END SOLUTION` lines.


In [ ]:
!pip install dafsa

In [ ]:
import itertools
from collections import defaultdict, Counter
from tqdm import tqdm
import networkx as nx
from dafsa import DAFSA
import re
import random

# Dataset prep

You think it's a bit wild (pun intended) that the records at this animal preserve are still all on paper. You digitize the zoologists' handwritten notes, and preprocess your dataset to include token indices of nouns that refer to animals in your nature preserve (ANIMAL), and token indices of other nouns detected by a spaCy dependency parser (NOUN).

Download this digitized file of pre-tagged notes by running the following cell.

In [ ]:
!wget https://raw.githubusercontent.com/lucy3/temp/main/tagged_sentences.txt

In this tab-delimited file, the first column is the lemmatized name of an animal, the second column is a sentence, and the third column is a tokenized, lemmatized form of that sentence. The remaining columns are indices of animals and non-target-animal nouns in the sentence.

In [ ]:
with open('tagged_sentences.txt', 'r') as infile:
  for line in infile:
    print(line.strip().split('\t')) # view each column
    break

Here are all animal *is-a* relations that you can remember right now. This is your set of seed tuples for bootstrapping.

In [ ]:
seed_tuples = [('kangaroo', 'marsupial'), ('kangaroo', 'mammal'), ('horse', 'mammal'), ('toucan', 'bird'),
                ('crocodile', 'reptile'), ('ant', 'anthropod'), ('dinosaur', 'reptile'), ('hawk', 'bird'),
                ('eagle', 'bird'), ('lark', 'bird'), ('toad', 'amphibian'), ('frog', 'amphibian'), ('ibis', 'bird'),
                ('oyster', 'shellfish'), ('pigeon', 'bird'), ('shark', 'fish'), ('hummingbird', 'bird'),
                ('jellyfish', 'cnidarian'), ('octopus', 'cephalopod'), ('anteater', 'xenarthan'), ('moose', 'mammal'),
                ('dragonfly', 'insect'), ('camel', 'mammal'), ('beetle', 'insect'), ('kinkajou', 'mammal'), ('alpaca', 'mammal')]

# Mining for patterns

Let's find sentences in the zoologists' notes that contain our seed set of `(ANIMAL, NOUN)` tuples, and extract the context around them.

Each phrasal pattern has three key parts:
- the middle context between one 'ANIMAL' and one 'NOUN'
- a left context of at most $n$ space-separated tokens to the left
- a right context of at most $m$ space-separated tokens to the right.

For example, here are some phrasal patterns with $n=2$ and $m=2$:

- `the ANIMAL be a NOUN .` ← "The giraffe is a mammal.",
- `as a NOUN , a ANIMAL have fur` ← "As a mammal, a giraffe has fur.".

### Deliverable 1 (40 pts ✨)

Implement the following function that takes in token indices of an `(ANIMAL, NOUN)` pair and a preprocessed sentence from your zoologists' notes. The output is a phrasal pattern.

For example, given the preprocessed (lemmatized & tokenized) sentence

```
['tall', 'and', 'long', '-', 'neck', ',', 'the', 'giraffe', 'be', 'a', 'mammal', ',', 'and', 'it', 'eat', 'leaf', 'from', 'high', 'branch', 'of', 'tree', '.']
```

and indices of `('giraffe', 'mammal')` as `(7, 10)`, and $n=2$ and $m=2$, your function should output

```
, the ANIMAL be a NOUN , and
```

With $n=5$ and $m=5$, your function should instead output

```
long - neck , the ANIMAL be a NOUN , and it eat leaf
```

In [ ]:
def transform_text_into_pattern(animal_noun, tokenized_sentence, n, m):
  '''
  @inputs:
  - animal_noun: a tuple of (ANIMAL, NOUN) indices
    For example, in the sentence "The horse is a domesticated, one-toed, hoofed mammal.",
    animal_noun == (1, 11),
    where token index 1 is "horse" and token index 11 is "mammal".
  - tokenized_sentence: a lemmatized, tokenized sentence, represented as a list of tokens
  - n: an integer indicating the size, or number of tokens, of the left context window
  - m: an integer indicating the size, or number of tokens, of the right content window

  @outputs:
  - phrase: a string containing left, middle, and right context around ANIMAL and NOUN
  '''
  phrase = ''

  ### BEGIN SOLUTION

  ### END SOLUTION

  return phrase.strip()

Check that your function works as expected by running the following cell.

In [ ]:
phrase = transform_text_into_pattern((1, 4),
      ['the', 'giraffe', 'be', 'a', 'mammal', '.'], 5, 5)
assert(phrase == 'the ANIMAL be a NOUN .')

phrase = transform_text_into_pattern((7, 10),
      ['tall', 'and', 'long', '-', 'neck', ',', 'the', 'giraffe', 'be', 'a',
       'mammal', ',', 'and', 'it', 'eat', 'leaf', 'from', 'high',
       'branch', 'of', 'tree', '.'], 5, 5)
assert(phrase == 'long - neck , the ANIMAL be a NOUN , and it eat leaf')

phrase = transform_text_into_pattern((7, 10),
      ['tall', 'and', 'long', '-', 'neck', ',', 'the', 'giraffe', 'be', 'a',
       'mammal', ',', 'and', 'it', 'eat', 'leaf', 'from', 'high',
       'branch', 'of', 'tree', '.'], 2, 2)
assert(phrase == ', the ANIMAL be a NOUN , and')

phrase = transform_text_into_pattern((5, 2),
      ['as', 'a', 'mammal', ',', 'a', 'giraffe', 'have', 'fur'], 5, 5)
assert(phrase == 'as a NOUN , a ANIMAL have fur')

Now, we call your function to extract phrasal patterns based on the seed set of tuples. To keep things tidy, we restrict the middle context to be $\leq 10$ tokens long.

In [ ]:
def parse_input_line(line):
  contents = line.strip().split('\t')
  animal = contents[0]
  sentence = contents[1]
  sentence_toks = contents[2].split(' ')

  # get animal and noun indices
  animal_idx = []
  noun_idx = []
  for i in range(3, len(contents)):
    if contents[i].startswith('ANIMAL'):
      animal_idx.append(int(contents[i].split()[-1]))
    elif contents[i].startswith('NOUN'):
      noun_idx.append(int(contents[i].split()[-1]))
  return animal, sentence, sentence_toks, animal_idx, noun_idx

def extract_phrasal_patterns(seed_tuples):
  '''
  @inputs:
  - seed_tuples: a set of seed tuples of (ANIMAL, NOUN)

  @outputs:
  - phrasal_patterns: a dictionary of patterns to their original sentences
  '''
  phrasal_patterns = defaultdict(list)

  with open('tagged_sentences.txt', 'r') as infile:
    for line in tqdm(infile, total=19587):
      animal, sentence, sentence_toks, animal_idx, noun_idx = parse_input_line(line)

      # get phrasal contexts of animal and noun pairs
      for pair in itertools.product(animal_idx, noun_idx):
        if (sentence_toks[pair[0]], sentence_toks[pair[1]]) in seed_tuples:
          if abs(pair[0] - pair[1]) <= 10:
            # calls the function that you wrote
            phrase = transform_text_into_pattern(pair, sentence_toks, 5, 5)
            phrasal_patterns[phrase].append(sentence)

  return phrasal_patterns

In [ ]:
phrasal_patterns = extract_phrasal_patterns(seed_tuples)

In [ ]:
pattern = 'the ANIMAL be a domesticate , one - toed , hoofed NOUN .'
print(phrasal_patterns[pattern])
# this cell should print out "['The horse is a domesticated, one-toed, hoofed mammal.']"

It is good to *generalize* the context around target entities, to account for irrelevant contextual variations. There are many ways one could approach this. Here, we'll replace infrequent parts of contexts with a wildcard `*`.

We implement pattern generalization for you. To do so, we construct a directed acyclic word graph by looping through all of the phrasal patterns our seed tuples found. Each edge of this graph represents one word (unlike [the example figure on Wikipedia](https://en.wikipedia.org/wiki/Deterministic_acyclic_finite_state_automaton), where each edge is a letter/character). Each path starting at the root of this graph is a phrasal pattern, and edge weights are incremented based on how often they are traversed across all patterns. If an edge appears less than 3 times, it is replaced with a `*`.  Our implementation is heavily inspired by Section 3.3 of [this paper](https://arxiv.org/abs/1804.02525).

For example, the sentences "_**Giraffes** are large African hoofed **mammals** who eats leaves._" and "_**Giraffes** are tall **mammals** living in grassy, open woodlands._" could be both represented by the pattern `ANIMAL be * NOUN *`.

In [ ]:
def build_graph(phrasal_patterns):
    """
    Builds a directed word graph from a set of phrasal patterns.
    Each word is weighted by its count in the graph.

    @input:

    @output:
    - a networkx graph
    """
    words = []
    for pattern in phrasal_patterns:
        words.append(pattern.split(' '))
    d = DAFSA(words) # creates directed acyclic word graphs

    # convert d into a networkx graph that is directed and has parallel edges
    graph = nx.MultiDiGraph()
    for node_id, node in d.nodes.items():
      graph.add_node(node_id)
      graph.nodes[node_id]["final"] = node.final

    for left in d.nodes.values():
        for label, right in left.edges.items():
            l_id = left.node_id
            r_id = right.node.node_id
            graph.add_edge(l_id, r_id, weight=right.weight, label=label)

    return graph

def clean_up_patterns(all_patterns):
  clean_patterns = set()
  for pattern in all_patterns:
    pattern_string = ' '.join(pattern)
    # reduce repeated * into a single *
    pattern_string = re.sub(r'(\*\s?)+', '* ', pattern_string)
    # constraint: some middle context must exist
    if 'ANIMAL * NOUN' in pattern_string or 'NOUN * ANIMAL' in pattern_string:
      continue
    clean_patterns.add(pattern_string.strip())
  clean_patterns = list(clean_patterns)
  return clean_patterns

def get_generalized_paths(graph):
  '''
  @inputs:
  - a networkx graph

  @outputs:
  - generalized phrasal patterns, as strings
  '''
  final_nodes = []
  for node in graph.nodes(data=True):
    if node[1]['final']:
      final_nodes.append(node[0])
  weight_threshold = 3 # number of times a transition/edge must exist for us to not wildcard it
  do_not_wildcard = ['ANIMAL', 'NOUN'] # tokens that shouldn't be replaced with a wildcard

  clustered_patterns = []
  for end in final_nodes:
    paths = nx.all_simple_edge_paths(graph, 0, end)
    for path in paths:
      this_path = []
      for edge in path:
          src = edge[0]
          dst = edge[1]
          edge_data = graph.get_edge_data(edge[0], edge[1])[edge[2]]
          if edge_data['label'] not in do_not_wildcard and \
              edge_data['weight'] < weight_threshold:
            this_path.append('*')
          else:
            this_path.append(edge_data['label'])
      clustered_patterns.append(this_path)

  clustered_patterns = clean_up_patterns(clustered_patterns)
  clustered_patterns = sorted(clustered_patterns)
  return clustered_patterns

In [ ]:
graph = build_graph(phrasal_patterns)
clustered_patterns = get_generalized_paths(graph)

In [ ]:
clustered_patterns

As a sanity check for your implementation, the above cell should output patterns containing n-grams such as "like other NOUN", "like most NOUN", "ANIMAL be", and "unlike other NOUN".

You may notice that some of these patterns are subsets of others. The extent to which we rely on `*` to be a catch-all for the context around ANIMAL and potential NOUNs affects precision, as we'll soon see.  

# Identify more animal-type tuples

Now, let's use these patterns to find more potential *is-a* relations for the animals in our preserve.

### Deliverable 2 (30 pts ✨)

Your task is to write a function that takes in a candidate phrase containing ANIMAL and NOUN, and checks whether it matches with a pattern we mined above. Hint: use the regex package `re`.

In [ ]:
def is_match(cand_phrase, pattern):
  '''
  @input:
  - cand_phrase: a candidate phrase containing ANIMAL and NOUN, e.g. "like other NOUN , the ANIMAL eat hay ."
  - pattern: a generalized pattern, e.g. "like other NOUN , * ANIMAL *"

  @output:
  - a boolean of True or False
  '''
  ### BEGIN SOLUTION

  ### END SOLUTION

Test out your implementation by running the following cell.

In [ ]:
output = is_match("like other NOUN , the ANIMAL eat hay .", "like other NOUN , * ANIMAL *")
assert output == True

output = is_match("like other NOUN , ANIMAL eat hay .", "like other NOUN , * ANIMAL *")
assert output == False

output = is_match("like other NOUN , ANIMAL eat hay .", "like other NOUN , ANIMAL *")
assert output == True

output = is_match("ANIMAL be a domesticate , one - toed , hoofed NOUN .", "ANIMAL be * NOUN *")
assert output == True

In [ ]:
def get_candidates():
  '''
  @outputs:
  - candidates: a list of (phrase, animal, noun), e.g. ('the ANIMAL be a domesticate , one - toed , hoofed NOUN .', 'horse', 'mammal')
  '''
  candidates = []
  with open('tagged_sentences.txt', 'r') as infile:
    for line in tqdm(infile, total=19587):
      animal, sentence, sentence_toks, animal_idx, noun_idx = parse_input_line(line)

      # get phrasal contexts of animal and noun pairs
      for pair in itertools.product(animal_idx, noun_idx):
        if abs(pair[0] - pair[1]) <= 10:
          # calls the function that you wrote
          phrase = transform_text_into_pattern(pair, sentence_toks, 5, 5)
          candidates.append((phrase, sentence_toks[pair[0]], sentence_toks[pair[1]]))
  return candidates

def filter_for_matches(candidates, clustered_patterns):
  matches = defaultdict(list)
  for cand in tqdm(candidates):
    for pattern in clustered_patterns:
      if is_match(cand[0], pattern):
        matches[pattern].append(cand)
  return matches

In [ ]:
candidates = get_candidates()
matches = filter_for_matches(candidates, clustered_patterns)

### Deliverable 3 (30 pts ✨)

You remember from Jurafsky & Martin Ch. 20, page 8 that
> Bootstrapping systems also assign confidence values to new tuples to avoid semantic drift. In semantic drift, an erroneous pattern leads to the introduction of erroneous tuples, which, in turn, lead to the creation of problematic patterns and the meaning of the extracted relations ‘drifts’.

> Confidence values for patterns are based on balancing two factors: the pattern’s performance with respect to the current set of tuples and the pattern’s productivity in terms of the number of matches it produces in the document collection.

With the above in mind, write the following function, `hits_and_finds()`, which returns the following:
- `num_hits`: the number of unique (ANIMAL, NOUN) tuples in our seed set (`seed_tuples`) that a pattern $p$ matches while looking in our dataset.
- `num_finds`: the total number of unique (ANIMAL, NOUN) tuples that $p$ finds in our dataset.

In [ ]:
def hits_and_finds(matches, p, seed_tuples):
  '''
  @inputs:
  - matches: a dictionary of the format {generalized pattern: [list of tuples (preprocessed sentence, animal, noun) ] }
  - p: a generalized pattern
  - seed_tuples: a list of seed tuples of the format (animal, noun)

  @outputs:
  - num_hits: an int, or the number of unique seed animal-type tuples found by p
  - num_finds: an int, or the total number of unique animal-type tuples found by p
  '''
  assert p in matches

  ### BEGIN SOLUTION

  ### END SOLUTION

  return num_hits, num_finds

Sanity check the implementation of your function with the following toy inputs.

In [ ]:
matches_toy = {
    'the ANIMAL be a * NOUN *' : [
              ('the ANIMAL be a domesticate , one - toed , hoofed NOUN .',
               'horse',
               'mammal'),
              ('the ANIMAL be a medium - sized NOUN species that be often slightly',
               'mallard',
               'waterfowl'),
              ('the ANIMAL be a social NOUN , form pack of two',
               'meerkat',
               'mammal'),
              ('the ANIMAL be a NOUN .',
               'horse',
               'mammal'),
              ]
}
p_toy = 'the ANIMAL be a * NOUN *'
seed_tuples_toy = [('horse', 'mammal'), ('mallard', 'waterfowl'), ('toucan', 'bird')]

num_hits, num_finds = hits_and_finds(matches_toy, p_toy, seed_tuples_toy)
assert num_hits == 2 # because we hit ('horse', 'mammal') and ('mallard', 'waterfowl')
assert num_finds == 3 # because we find ('horse', 'mammal'), ('mallard', 'waterfowl'), and ('meerkat', 'mammal')

Let's inspect phrasal patterns and the `(ANIMAL, NOUN)` tuples they surface. For each pattern, we'll give examples of both hits and *new* finds.

In [ ]:
def show_results(matches, seed_tuples):
  '''
  Prints candidate animal-type tuples and the patterns
  that identified them.
  '''
  for pattern in sorted(matches.keys()):
    num_hits, num_finds = hits_and_finds(matches, pattern, seed_tuples)
    hits = []
    new_finds = []
    for tup in matches[pattern]:
      animal_type = (tup[1], tup[2])
      if animal_type in seed_tuples:
        hits.append(tup)
      else:
        new_finds.append(tup)
    if len(new_finds) > 20:
      new_finds = random.sample(new_finds, 20)

    random.seed(0)
    print("Pattern:", pattern)
    print("# of hits: ", num_hits)
    print("Hits:")
    for tup in hits:
      print('\t', tup[1], 'is a', tup[2], '←', tup[0])
    print("# of new finds: ", num_finds - num_hits)
    print("Examples of new finds:")
    if len(new_finds) == 0:
      print("\tNone")
    for tup in new_finds:
      print('\t', tup[1], 'is a', tup[2], '←', tup[0])
    print('\n---------------------------------\n')

Scroll through the output of the following cell.

In [ ]:
show_results(matches, seed_tuples)

💡 Think to yourself: when inspecting the new finds of each pattern, what do you notice?
- What patterns are productive, but imprecise?
- What patterns are precise, but unproductive?

In practice, one could run multiple iterations of this bootstrapping process to gather even more patterns and animal-type tuples. Confidence thresholds can help prevent semantic drift that arises from imprecise patterns.

Outside on the animal preserve, the sun is setting, and the animals are quieting down for nighttime. There's some promise here, but there's also some sense that you could refine your approach.

💡 Think to yourself:
- Is there a better way than string/phrase matching to capture patterns such as "*This animal preserve includes __mammals__ such as __lions__, __elephants__, and __giraffes__*"? (This [paper](https://papers.nips.cc/paper_files/paper/2004/hash/358aee4cc897452c00244351e4d91f69-Abstract.html) may contain clues.)
- You realize that there are some sentences in the zoologists' notes that are completely missed by your pipeline, such as the second sentence here: "*__Giraffes__ are tall. They are __mammals__.*" What are some ways you might improve your current approach or dataset?
- How might you extract not just ANIMAL → NOUN relations (e.g. "kangaroo" → "marsupial", "kangaroo" → "macropod"), but also NOUN → NOUN relations (e.g. "macropod" → "marsupial") to create a full taxonomy tree?

# Closing and Submission

Congratulations on finishing HW5! Please ensure that you submit this completed notebook onto Gradescope. Make sure all cells in the notebook are run so that print statements are visible.

File --> Download --> Download .ipynb

Make sure your notebook is named `HW5.ipynb` when you upload it to Gradescope.

# Addendum

The list of animals in this homework are drawn from [Wikipedia's list of animal names](https://en.wikipedia.org/wiki/List_of_animal_names), and the zoologists' notes are actually sentences pulled from Wikipedia.